In [1]:
import os
import glob
import psycopg2 
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=DataEng007")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
            
    return all_files 

song_files = get_files("./data/song_data")

filepath = song_files[0]
print(filepath)

/home/srivathsan/harini/playground_udacity/data-engineering/playground/input/Data_Modeling/Project 1/data/song_data/A/A/A/TRAAABD128F429CF47.json


In [4]:
df = pd.read_json(filepath, lines= True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


In [5]:
# artists table
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values.tolist()
artist_data

[['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892]]

In [6]:
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


In [7]:
artist_table_insert

'INSERT INTO artists (artist_id, name, location, lattitude, longitude)\nVALUES (%s, %s, %s, %s, %s) ON CONFLICT DO NOTHING;'

In [8]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [9]:
#Songs Table
song_data = df[["song_id", "title", "artist_id","year", "duration"]].values[0].tolist()
song_data

['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546]

In [10]:
#Insert record into table
song_table_insert

'INSERT INTO songs (song_id, title, artist_id, year, duration)\nVALUES (%s, %s, %s, %s, %s) ON CONFLICT DO NOTHING;'

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [12]:
#Process Log Files
log_files = get_files("./data/log_data")

In [13]:
filepath = log_files[0]
print(filepath)

/home/srivathsan/harini/playground_udacity/data-engineering/playground/input/Data_Modeling/Project 1/data/log_data/2018/11/2018-11-21-events.json


In [14]:
df = pd.read_json(filepath, lines = True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,741,None,307,1542760054796,None,
1,None,Logged In,Theodore,M,1,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,741,None,200,1542760086796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
2,None,Logged Out,None,None,0,None,NaN,paid,None,GET,Home,NaN,774,None,200,1542761399796,None,
3,None,Logged Out,None,None,1,None,NaN,paid,None,GET,Home,NaN,774,None,200,1542761485796,None,
4,None,Logged Out,None,None,2,None,NaN,paid,None,PUT,Login,NaN,774,None,307,1542761486796,None,


In [15]:

df = df.query("page == 'NextSong'")
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
6,Facto Delafe y las flores azules,Logged In,Tegan,F,4,Levine,315.81995,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Enero en la playa,200,1542761878796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
7,Kings Of Leon,Logged In,Kate,F,0,Harrell,204.27710,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Manhattan,200,1542761921796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
8,Franz Ferdinand,Logged In,Kate,F,1,Harrell,204.12036,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Michael,200,1542762125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
9,Blue October,Logged In,Tegan,F,5,Levine,272.32608,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Drilled A Wire Through My Cheek,200,1542762193796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
11,Elisa,Logged In,Kate,F,2,Harrell,248.97261,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Almeno Tu Nell'Universo,200,1542762329796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


In [16]:
#Time Table
t = pd.to_datetime(df['ts'], unit = 'ms')
df['ts'] = pd.to_datetime(df['ts'], unit='ms')

In [17]:
t.dt.hour.head()

6     0
7     0
8     1
9     1
11    1
Name: ts, dtype: int64

In [18]:
df['ts'].head()

6    2018-11-21 00:57:58.796
7    2018-11-21 00:58:41.796
8    2018-11-21 01:02:05.796
9    2018-11-21 01:03:13.796
11   2018-11-21 01:05:29.796
Name: ts, dtype: datetime64[ns]

In [19]:
time_data = list((t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday))
column_labels = list(('start_time', 'hour', 'day','week', 'month', 'year', 'weekday'))

In [20]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
6,2018-11-21 00:57:58.796,0,21,47,11,2018,2
7,2018-11-21 00:58:41.796,0,21,47,11,2018,2
8,2018-11-21 01:02:05.796,1,21,47,11,2018,2
9,2018-11-21 01:03:13.796,1,21,47,11,2018,2
11,2018-11-21 01:05:29.796,1,21,47,11,2018,2


In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [22]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
6,Facto Delafe y las flores azules,Logged In,Tegan,F,4,Levine,315.81995,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Enero en la playa,200,2018-11-21 00:57:58.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
7,Kings Of Leon,Logged In,Kate,F,0,Harrell,204.27710,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Manhattan,200,2018-11-21 00:58:41.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
8,Franz Ferdinand,Logged In,Kate,F,1,Harrell,204.12036,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Michael,200,2018-11-21 01:02:05.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
9,Blue October,Logged In,Tegan,F,5,Levine,272.32608,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Drilled A Wire Through My Cheek,200,2018-11-21 01:03:13.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
11,Elisa,Logged In,Kate,F,2,Harrell,248.97261,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Almeno Tu Nell'Universo,200,2018-11-21 01:05:29.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


In [23]:
#users Table
user_df = df[["userId", "firstName", "lastName","gender", "level"]]
user_df = user_df.drop_duplicates()

In [24]:
user_table_insert

'INSERT INTO users (user_id, first_name, last_name, gender, level)\nVALUES (%s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET level = EXCLUDED.level;'

In [25]:
#Insert records in user table
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [26]:
songplay_table_insert

'INSERT INTO songplays ( start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)\nVALUES (%s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT(songplay_id) DO NOTHING;'

In [27]:
#Songplays Table
song_select

'SELECT ss.song_id, ss.artist_id FROM songs ss JOIN artists ars on (ss.artist_id = ars.artist_id) WHERE ss.title = %s AND ars.name = %s AND ss.duration = %s;'

In [28]:
songplay_table_insert

'INSERT INTO songplays ( start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)\nVALUES (%s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT(songplay_id) DO NOTHING;'

In [29]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
6,Facto Delafe y las flores azules,Logged In,Tegan,F,4,Levine,315.81995,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Enero en la playa,200,2018-11-21 00:57:58.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
7,Kings Of Leon,Logged In,Kate,F,0,Harrell,204.27710,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Manhattan,200,2018-11-21 00:58:41.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
8,Franz Ferdinand,Logged In,Kate,F,1,Harrell,204.12036,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Michael,200,2018-11-21 01:02:05.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
9,Blue October,Logged In,Tegan,F,5,Levine,272.32608,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Drilled A Wire Through My Cheek,200,2018-11-21 01:03:13.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
11,Elisa,Logged In,Kate,F,2,Harrell,248.97261,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Almeno Tu Nell'Universo,200,2018-11-21 01:05:29.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


In [30]:
for index, row in df.iterrows():
    #get songId & artistId from song & artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    print(songid, artistid) 
     #insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit() 
    
   

None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None


In [31]:
conn.close()